# Probability-Probability tests - Figures 4 & B1-3

In [ ]:
import h5py
from pathlib import Path
from gw_smc_utils.plotting import set_style, pp_plot_from_credible_levels
import pandas as pd
import matplotlib.pyplot as plt
import re

set_style()

Path("figures").mkdir(exist_ok=True)

In [ ]:

data_release_path = Path("../data_release/gw_smc_data_release_results/")
credible_levels_path = data_release_path / "simulated_data" / "pp_tests" / "credible_levels"


In [ ]:
keys = [
    "chirp_mass",
    "mass_ratio",
    "a_1",
    "a_2",
    "tilt_1",
    "tilt_2",
    "phi_12",
    "phi_jl",
    "luminosity_distance",
    "dec",
    "ra",
    "theta_jn",
    "psi",
    "geocent_time",
    "phase",
]

In [ ]:
credible_level_files = sorted(credible_levels_path.glob("*.hdf5"))
for clf in credible_level_files:
    sampler = re.search(r"(dynesty|pocomc)", clf.name).group(0)
    ndet = re.search(r"(\d+)det", clf.name).group(0)
    print(f"\nProducing P-P plot for {sampler} in {ndet}")
    if not sampler or not ndet:
        raise ValueError(f"Could not extract sampler or ndet from {clf.name}")
    credible_levels = pd.read_hdf(clf, "credible_levels")
    fig, pvalues = pp_plot_from_credible_levels(
        credible_levels,
    )
    fig.savefig(f"figures/pp_test_{sampler}_{ndet}.pdf")
    plt.show()